In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [ ]:
labels=pd.read_csv("/kaggle/input/severstal-steel-defect-detection/train.csv")

In [ ]:
labels.head()

In [ ]:
labels.head()

In [ ]:
labels["ClassId"].value_counts()

In [ ]:
len(labels["ImageId"])

In [ ]:
from tqdm import tqdm
import cv2

In [ ]:
x=[]  #### variable to store all steel images
z=[] #### # variable to store all  images ClassId
for img,ClassId,EncodedPixels in tqdm(labels.values):
    image=cv2.imread("/kaggle/input/severstal-steel-defect-detection/train_images/{}".format(img),cv2.IMREAD_COLOR)
    image=cv2.resize(image,(120,120))
    x.append(image)
    z.append(ClassId)
   

In [ ]:
index=10
plt.imshow(x[index])

In [ ]:
index=11
plt.imshow(x[index])

In [ ]:
index=12
plt.imshow(x[index])

In [ ]:
index=20
plt.imshow(x[index])

In [ ]:
index=21
plt.imshow(x[index])

In [ ]:
index=25
plt.imshow(x[index])

In [ ]:
from keras.utils import to_categorical

In [ ]:
encdr=LabelEncoder()
y=encdr.fit_transform(z)
y=to_categorical(y)

In [ ]:
x=np.array(x)
x=x/255

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,shuffle=True)

In [ ]:
print("x_train shape:",x_train.shape)
print("x_test shape:",x_test.shape)
print("y_train shape:",y_train.shape)
print("y_test shape:",y_test.shape)

In [ ]:
# Defining and training the model

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(120,120,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(256,activation="relu"))
model.add(Dense(4,activation="softmax"))
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adam(),
             metrics=["accuracy"])


In [ ]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test),verbose=1)

In [ ]:
### Using VGG-19 for training the model

In [ ]:
from keras.applications.vgg19 import VGG19

In [ ]:
VGG19_model=VGG19(include_top=False,weights="imagenet",input_shape=(120,120,3),pooling="max")
VGG19_model.summary()
VGG19_model.trainable=False

In [ ]:
model_vg19=Sequential()
model_vg19.add(VGG19_model)
model_vg19.add(Flatten())
model_vg19.add(Dense(128,activation="relu"))
model_vg19.add(Dropout(0.3))
model_vg19.add(Dense(256,activation="relu"))
model_vg19.add(Dropout(0.3))
model_vg19.add(Dense(256,activation="relu"))
model_vg19.add(Dense(4,activation="softmax"))

In [ ]:
model_vg19.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(),
                  metrics=["accuracy"])

In [ ]:
model_vg19.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test),verbose=1)

In [ ]:
### Training the model using ResNet50

In [ ]:
from keras.applications.resnet50 import ResNet50

In [ ]:
ResNet50_model=ResNet50(include_top=False,weights="imagenet",input_shape=(120,120,3),pooling="max")
ResNet50_model.summary()
ResNet50_model.trainable=False

In [ ]:
ResNet50_new_model=Sequential()
ResNet50_new_model.add(ResNet50_model)
ResNet50_new_model.add(Flatten())
ResNet50_new_model.add(Dense(128,activation="relu"))
ResNet50_new_model.add(Dropout(0.5))
ResNet50_new_model.add(Dense(256,activation="relu"))
ResNet50_new_model.add(Dropout(0.5))
ResNet50_new_model.add(Dense(256,activation="relu"))
ResNet50_new_model.add(Dense(4,activation="softmax"))

In [ ]:
ResNet50_new_model.compile(loss=keras.losses.categorical_crossentropy,
                          optimizer=keras.optimizers.Adam(),
                          metrics=["accuracy"])
ResNet50_new_model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test),verbose=1)